In [6]:
import pandas as pd
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import re
import numpy as np

In [7]:
# 讀取Raw資料
def loadCsvFileToDict(filename, ueid):
    # 读取CSV文件
    data = pd.read_csv(filename)

    headers = data.columns.tolist()
    # print("headers in cell csv = ",headers)

    data_dict = {header: [] for header in headers}

    if ueid != 'null' :
        ueid_rows = data[data['UeID'] == ueid].reset_index(drop=True)
    else :
        ueid_rows = data
    
    for _, row in ueid_rows.iterrows():
        for header, value in row.items():
            if str(value).isdigit():
                data_dict[header].append(int(value))
            elif re.match(r'^[-+]?\d*\.?\d*$', str(value)):
                data_dict[header].append(float(value))
            elif re.match(r'^S.*$', str(value)):
                # print(value)
                data_dict[header].append(int(value[1]))
            # else :
            #     print(row)
            #     print("loadCsvFileToDict have an unknown value : ", value)

    # Access the processed data
    # for header, values in data_dict.items():
    #     print(header, values)

    #print("example to show clown [Viavi.Geo.x] = ",data['Viavi.Geo.x'])
    print("load " + filename + " successed")
    return data_dict

def PrepareInput(ue_data):
    unmerge_data = {
        "Timestamp" : ue_data['Timestamp'],
        "ServingCellId": ue_data['ServingCellId'],
        "ServingCellRsrp": ue_data['ServingCellRsrp'],
        #"ServingCellRsSinr": ue_data['ServingCellRsSinr'],
        "neighbourCell1": ue_data['neighbourCell1'],
        "neighbourCell1Rsrp": ue_data['neighbourCell1Rsrp'],
        #"neighbourCell1RsSinr": ue_data['neighbourCell1RsSinr'],
        "neighbourCell2": ue_data['neighbourCell2'],
        "neighbourCell2Rsrp": ue_data['neighbourCell2Rsrp'],
        #"neighbourCell2RsSinr": ue_data['neighbourCell2RsSinr'],
        "neighbourCell3": ue_data['neighbourCell3'],
        "neighbourCell3Rsrp": ue_data['neighbourCell3Rsrp'],
        #"neighbourCell3RsSinr": ue_data['neighbourCell3RsSinr'],
        "neighbourCell4": ue_data['neighbourCell4'],
        "neighbourCell4Rsrp": ue_data['neighbourCell4Rsrp'],
        #"neighbourCell4RsSinr": ue_data['neighbourCell4RsSinr'],
        "neighbourCell5": ue_data['neighbourCell5'],
        "neighbourCell5Rsrp": ue_data['neighbourCell5Rsrp'],
        #"neighbourCell5RsSinr": ue_data['neighbourCell5RsSinr'],
    }
    unmerge_data_list = list(unmerge_data.items())
    print(unmerge_data_list[1][0])
    print(unmerge_data_list[1][1])
    # print(unmerge_data_list[1][1][1])
    serving_cell_id = unmerge_data_list[1][1]

    time = unmerge_data_list[0][1]
    cell_id = np.zeros((4000, 6))
    cell_id_index = [1,3,5,7,9,11]
    cell_rsrp = np.zeros((4000, 6))


    for time in range (len(time)):
        serving_id_temp = unmerge_data_list[1][1][time]
        cell_id[time][serving_id_temp - 1] = 1
        
        for cell_num in cell_id_index:
            cell_num_temp = unmerge_data_list[cell_num][1][time]
            cell_rsrp[time][cell_num_temp - 1] = unmerge_data_list[cell_num + 1][1][time]
            
    # print(cell_id)

    # 處理重複點
    zero_points = np.argwhere(cell_rsrp == 0)
    # print(zero_points)
    for zero_points_index in range(len(zero_points)):
        time = zero_points[zero_points_index][0]
        cell = zero_points[zero_points_index][1]
        # 前兩點的插值
        last_two_point_diff = cell_rsrp[time-2][cell] - cell_rsrp[time-1][cell]
        consider_last_two_point = cell_rsrp[time-1][cell] - last_two_point_diff
        # 後兩點的插值
        future_two_point_diff = cell_rsrp[time+2][cell] - cell_rsrp[time+1][cell]
        consider_future_two_point = cell_rsrp[time+1][cell] - future_two_point_diff
        # 隔壁鄰居的數值 12
        avg_neighbor = sum(cell_rsrp[time])/5
        if(abs(avg_neighbor - consider_last_two_point) < 12):
            cell_rsrp[time][cell] = consider_last_two_point
        if(abs(avg_neighbor - consider_future_two_point) < 12):
            cell_rsrp[time][cell] = consider_future_two_point
    
    return time, cell_id, cell_rsrp, serving_cell_id

In [8]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(6, 24)
        self.fc2 = nn.Linear(24, 48)
        self.fc3 = nn.Linear(48, 6)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
ue_csv_file = 'output_ue.csv'
ue_data = loadCsvFileToDict(ue_csv_file, 1)
time, cell_id, cell_rsrp, serving_cell_id = PrepareInput(ue_data)

load output_ue.csv successed
ServingCellId
[3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 創建模型實例並加載權重
model = DQN().to(device)
model.load_state_dict(torch.load('ddqn_model_weights.pth'))
model.eval()

# 假設你的預測數據是一個名為data的列表
data = cell_rsrp  # 這裡的...代表你的數據

# 將你的數據轉換為PyTorch張量
tensor = torch.tensor(data, dtype=torch.float32)

# 確保張量在與模型同一設備上
tensor = tensor.to(device)

# 獲取預測結果
predictions = model(tensor)

# 如果你需要將預測結果轉換回NumPy數組，可以這樣做：
predictions = predictions.cpu().detach().numpy()

# 現在，predictions就是模型對你的數據的預測結果

In [12]:
print(predictions)
print(len(predictions))

[[  0.48970792 -47.900864   -50.642967    -0.5105429    0.48665503
    2.8127205 ]
 [  0.48970792 -47.900864   -50.642967    -0.5105429    0.48665503
    2.8127205 ]
 [  0.48970792 -47.900864   -50.642967    -0.5105429    0.48665503
    2.8127205 ]
 ...
 [  0.48970792 -47.900864   -50.642967    -0.5105429    0.48665503
    2.8127205 ]
 [  0.48970792 -47.900864   -50.642967    -0.5105429    0.48665503
    2.8127205 ]
 [  0.48970792 -47.900864   -50.642967    -0.5105429    0.48665503
    2.8127205 ]]
